# 데이터 준비와 전처리

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/Ex_14/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/Ex_14/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# 평점과 무비 데이터프레임 합치기
data = ratings.join(movies.set_index('movie_id'), on='movie_id')
data.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [7]:
# 사용하지 않는 컬럼 제거
data = data.drop(columns=['timestamp', 'genre'])
data.head()

,user_id,movie_id,counts,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


# 데이터 분석

In [8]:
# data에 있는 영화 개수
data['movie_id'].nunique()

3628

In [9]:
# data에 있는 사용자 수
data['user_id'].nunique()

6039

In [10]:
# 가장 인기 있는 영화 30개
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [11]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

# 내가 선호하는 영화 5가지 골라서 ratings에 추가

In [12]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('super', regex=False)]

,movie_id,title,genre
542,546,Super Mario Bros. (1993),Action|Adventure|Children's|Sci-Fi
645,651,"Superweib, Das (1996)",Comedy
850,861,Supercop (1992),Action|Thriller
2571,2640,Superman (1978),Action|Adventure|Sci-Fi
2572,2641,Superman II (1980),Action|Adventure|Sci-Fi
2573,2642,Superman III (1983),Action|Adventure|Sci-Fi
2574,2643,Superman IV: The Quest for Peace (1987),Action|Adventure|Sci-Fi
2838,2907,Superstar (1999),Comedy
3121,3190,Supernova (2000),Adventure|Sci-Fi
3807,3877,Supergirl (1984),Action|Adventure|Fantasy


In [13]:
# 내가 좋아하는 영화 5편의 id와 title 리스트 만들기
my_favorite_id = [2640, 2641, 2642, 2643, 3877]

my_favorite_title = []
for i in my_favorite_id:
    my_favorite_title.extend(list(movies[movies['movie_id'] == i]['title']))

# 'YBG'라는 user_id가 위 5편의 영화를 5번씩 봤다고 가정
my_movielist = pd.DataFrame({'user_id': ['YBG']*5, 'movie_id': my_favorite_id, 'counts': [5]*5, 'title': my_favorite_title})
my_movielist

,user_id,movie_id,counts,title
0,YBG,2640,5,Superman (1978)
1,YBG,2641,5,Superman II (1980)
2,YBG,2642,5,Superman III (1983)
3,YBG,2643,5,Superman IV: The Quest for Peace (1987)
4,YBG,3877,5,Supergirl (1984)


In [14]:
if not data.isin({'user_id':['YBG']})['user_id'].any():  # user_id에 'YBG'라는 데이터가 없다면
    data = data.append(my_movielist, ignore_index=True)  # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)  # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,YBG,2640,5,Superman (1978)
836479,YBG,2641,5,Superman II (1980)
836480,YBG,2642,5,Superman III (1983)
836481,YBG,2643,5,Superman IV: The Quest for Peace (1987)
836482,YBG,3877,5,Supergirl (1984)


# CSR matrix

## indexing

In [15]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저 id와 영화 제목을 indexing 하는 코드
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx  = {v:k for k,v in enumerate(movie_unique)}

In [16]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx를 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title  column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('title  column indexing Fail!!')

data

user_id column indexing OK!!
title  column indexing OK!!


,user_id,movie_id,counts,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,2640,5,526
836479,6039,2641,5,763
836480,6039,2642,5,1534
836481,6039,2643,5,848


## CSR_matrix

In [17]:
from scipy.sparse import csr_matrix

# CSR Matrix 생성
num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data['counts'], (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# als_model = AlternatingLeastSquares 모델 구성 및 훈련

In [18]:
from implicit.als import AlternatingLeastSquares
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분이며 학습 내용과는 무관
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [19]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [20]:
# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [21]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도 파악

In [22]:
# 훈련된 모델이 만든 나의 벡터와 영화 벡터 구하기
YBG, matrix = user_to_idx['YBG'], movie_to_idx['Superman (1978)']
YBG_vector, matrix_vector = als_model.user_factors[YBG], als_model.item_factors[matrix]

In [23]:
YBG_vector

array([ 0.00691993, -0.07688189, -0.4487717 ,  0.26892707,  0.4594067 ,
       -0.02580609,  0.09802366, -0.07386509,  0.03423259, -0.20111823,
       -0.29446417,  0.41989312, -0.26036355, -0.17096171,  0.19722931,
        0.42028162, -0.12283732, -0.9595671 ,  0.5510436 , -0.5888675 ,
       -0.10998223, -0.2892762 , -0.00275824,  0.08106063, -0.81017745,
       -0.04599769,  0.04217477,  0.5331792 , -0.3294944 , -0.45822558,
        0.13518073, -0.5132814 , -0.33158576,  0.46357733,  0.81460243,
        0.06370334, -0.30258143,  0.25789383, -0.07476836, -0.3575456 ,
       -0.5733679 ,  0.02353976,  0.00868759,  0.63990736, -0.03415733,
       -0.41468737, -0.00340584, -0.20338345, -0.6916908 ,  0.6114675 ,
        0.5225921 ,  0.63325053,  0.4239714 ,  0.6205674 ,  0.6662335 ,
        0.2363485 ,  0.5885362 , -0.01018897,  0.21334209,  0.37142876,
        0.06161496, -0.24239396,  0.19516398,  0.4060139 , -0.03580529,
        0.03297867, -0.25685498, -0.9367216 , -0.09630204, -0.28

In [24]:
matrix_vector

array([ 0.01362601,  0.00914477, -0.01079823,  0.00490635,  0.01996235,
        0.01205686,  0.00537024,  0.01814869,  0.01708662, -0.00192149,
        0.00132707,  0.00345287,  0.00120972,  0.01286158,  0.00713236,
        0.02836559, -0.01034252, -0.01207602,  0.01341706, -0.00673345,
        0.0004112 , -0.00470125, -0.02467289, -0.00087684, -0.016508  ,
        0.01463589, -0.00538814,  0.00690436,  0.01829741, -0.00485228,
       -0.00690756, -0.00322261, -0.00479295,  0.01058535,  0.02114647,
        0.01161516,  0.00182325,  0.0159079 ,  0.00410516, -0.01498139,
       -0.00305785,  0.00307008,  0.0040148 ,  0.0225046 ,  0.00745245,
       -0.00326522,  0.00324176, -0.00980347, -0.01398553,  0.0318439 ,
        0.02910771,  0.00650995,  0.01480007,  0.02426173,  0.01190238,
        0.00669425,  0.03144727,  0.0062201 ,  0.01317723,  0.0277121 ,
        0.0053572 ,  0.0002444 ,  0.0075594 ,  0.02383054,  0.0114051 ,
        0.01239879,  0.00145354, -0.02498011,  0.01480451, -0.00

In [25]:
# 나와 매트릭스의 내적 구하기 (선호 리스트에 있는 영화)
np.dot(YBG_vector, matrix_vector)

0.45114127

In [26]:
# 다른 영화에 대한 선호도 예측 (선호 리스트에 없는 영화)
matrix = movie_to_idx['My Fair Lady (1964)']
matrix_vector = als_model.item_factors[matrix]
np.dot(YBG_vector, matrix_vector)

-0.002497015

# 내가 좋아하는 영화와 비슷한 영화 추천받기

In [27]:
favorite_movie = 'Superman (1978)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie  # (movie idx, 유사도) Tuple

[(526, 1.0000001),
 (763, 0.7962219),
 (581, 0.6188888),
 (928, 0.5830554),
 (777, 0.5660079),
 (912, 0.56445),
 (704, 0.55415934),
 (584, 0.5463975),
 (586, 0.5303016),
 (1534, 0.51578015),
 (188, 0.5150998),
 (893, 0.50530434),
 (527, 0.50288635),
 (685, 0.49495617),
 (615, 0.4802251)]

In [28]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Superman (1978)',
 'Superman II (1980)',
 'Star Trek: The Motion Picture (1979)',
 'Robocop (1987)',
 "Logan's Run (1976)",
 'Westworld (1973)',
 'Tron (1982)',
 'Star Trek III: The Search for Spock (1984)',
 'Star Trek IV: The Voyage Home (1986)',
 'Superman III (1983)',
 'Indiana Jones and the Temple of Doom (1984)',
 'Stepford Wives, The (1975)',
 'Batman (1989)',
 'Planet of the Apes (1968)',
 'Cocoon (1985)']

In [29]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [30]:
get_similar_movie('Supergirl (1984)')

['Supergirl (1984)',
 'Cocoon: The Return (1988)',
 'Heart Condition (1990)',
 'Twice Upon a Yesterday (1998)',
 'Superman IV: The Quest for Peace (1987)',
 'Gnome-Mobile, The (1967)',
 'Slumber Party Massacre III, The (1990)',
 'Prom Night IV: Deliver Us From Evil (1992)',
 'Benji the Hunted (1987)',
 'Deadly Friend (1986)']

# 내가 가장 좋아할 만한 영화들을 추천받기

In [31]:
user = user_to_idx['YBG']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(581, 0.30726802),
 (584, 0.3060841),
 (586, 0.27679637),
 (1164, 0.26379448),
 (928, 0.25523168),
 (704, 0.25240275),
 (583, 0.24321586),
 (985, 0.22501379),
 (674, 0.22097632),
 (884, 0.20804933),
 (527, 0.2056588),
 (499, 0.1946086),
 (113, 0.19174641),
 (188, 0.18189076),
 (1704, 0.17047723),
 (558, 0.16731603),
 (913, 0.16697526),
 (777, 0.16595204),
 (699, 0.16267928),
 (1077, 0.16160974)]

In [32]:
# 인덱스를 영화 제목으로 변환
[idx_to_movie[i[0]] for i in movie_recommended]

['Star Trek: The Motion Picture (1979)',
 'Star Trek III: The Search for Spock (1984)',
 'Star Trek IV: The Voyage Home (1986)',
 'Star Trek V: The Final Frontier (1989)',
 'Robocop (1987)',
 'Tron (1982)',
 'Star Trek: The Wrath of Khan (1982)',
 'Running Man, The (1987)',
 'Back to the Future Part II (1989)',
 'Mad Max Beyond Thunderdome (1985)',
 'Batman (1989)',
 'Honey, I Shrunk the Kids (1989)',
 'Star Trek VI: The Undiscovered Country (1991)',
 'Indiana Jones and the Temple of Doom (1984)',
 'Karate Kid, Part II, The (1986)',
 'Star Trek: Generations (1994)',
 'Fly, The (1986)',
 "Logan's Run (1976)",
 'Star Trek: Insurrection (1998)',
 'Transformers: The Movie, The (1986)']

In [34]:
# 사용자가 기록을 남긴 데이터가 특정 영화 추천에 기여한 정도를 확인
recommended = movie_to_idx['Star Trek: The Motion Picture (1979)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Superman (1978)', 0.10227421440560774),
 ('Superman II (1980)', 0.09693673095429117),
 ('Superman III (1983)', 0.05777196803606697),
 ('Superman IV: The Quest for Peace (1987)', 0.03379795400029876),
 ('Supergirl (1984)', 0.010857294642872068)]

# 회고

- 추천시스템을 직접 실습해보아 재미있었고 뭔가 최신의 데이터셋으로 해보았으면 어떨까도 싶었다.
- 데이터톤에서 넷플릭스 데이터를 골랐기에 이를 활용해 볼 수도 있겠다고 생각했다.
- 추천해준 영화의 관계성이 나름 있어보여 결과가 나쁘지 않은 것 같다.
- 조금 더 모델 훈련을 깊게 해보면 좋을 것 같고 이외 장르와 함께도 생각해보면 좋을 것 같다.
- 영화 선택에 있어서도 시리즈물로 너무 일관성이 있게 하여 살짝은 다른 각각의 영화를 골라보면 어떨지도 생각해봐야 겠다.